# Analyze Change Detection Output

This notebook helps understand the change detection results by showing the underlying chunks and changes.

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

DATABASE_DIR = Path.cwd().parent / "databases"
DB_PATH = DATABASE_DIR / "my_database.db"

print(f"Database: {DB_PATH}")
print(f"Exists: {DB_PATH.exists()}")

## 1. Show All Chunks for Employee_Handbook.pdf

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    chunks_df = pd.read_sql_query("""
        SELECT
            cr.raw_file_nme,
            cr.raw_file_version_nbr,
            cc.chunk_id,
            cc.chunk_index,
            cc.content_checksum,
            LENGTH(cc.chunk_text) as chunk_length,
            SUBSTR(cc.chunk_text, 1, 200) as text_preview
        FROM content_chunks cc
        JOIN content_repo cr ON cc.ud_source_file_id = cr.ud_source_file_id
        WHERE cr.raw_file_nme = 'Employee_Handbook.pdf'
          AND cc.status = 'active'
        ORDER BY cr.raw_file_version_nbr, cc.chunk_index
    """, conn)

print(f"\nTotal chunks for Employee_Handbook.pdf: {len(chunks_df)}\n")
display(chunks_df)

## 2. Show Full Text of Each Chunk

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    chunks_full = pd.read_sql_query("""
        SELECT
            cr.raw_file_nme,
            cr.raw_file_version_nbr,
            cc.chunk_index,
            cc.content_checksum,
            cc.chunk_text
        FROM content_chunks cc
        JOIN content_repo cr ON cc.ud_source_file_id = cr.ud_source_file_id
        WHERE cr.raw_file_nme = 'Employee_Handbook.pdf'
          AND cc.status = 'active'
        ORDER BY cr.raw_file_version_nbr, cc.chunk_index
    """, conn)

for idx, row in chunks_full.iterrows():
    print("="*100)
    print(f"File: {row['raw_file_nme']} | Version: {row['raw_file_version_nbr']} | Chunk: {row['chunk_index']}")
    print(f"Checksum: {row['content_checksum']}")
    print("="*100)
    print(row['chunk_text'])
    print("\n")

## 3. Show Changes Detected

In [ ]:
# Get the most recent detection run
with sqlite3.connect(DB_PATH) as conn:
    runs_df = pd.read_sql_query("""
        SELECT DISTINCT detection_run_id, created_dt
        FROM content_change_log
        ORDER BY created_dt DESC
        LIMIT 5
    """, conn)
    
print("Recent detection runs:")
display(runs_df)

if len(runs_df) > 0:
    latest_run = runs_df.iloc[0]['detection_run_id']
    print(f"\nShowing results for: {latest_run}")

In [ ]:
# Show all changes for Employee_Handbook.pdf
with sqlite3.connect(DB_PATH) as conn:
    changes_df = pd.read_sql_query("""
        SELECT 
            file_name,
            change_type,
            current_checksum,
            previous_checksum,
            similarity_score,
            SUBSTR(current_content, 1, 200) as current_preview,
            SUBSTR(previous_content, 1, 200) as previous_preview
        FROM content_change_log
        WHERE detection_run_id = ?
          AND file_name = 'Employee_Handbook.pdf'
        ORDER BY change_type
    """, conn, params=[latest_run])

print(f"\nChanges detected for Employee_Handbook.pdf: {len(changes_df)}\n")
display(changes_df)

## 4. Show Full Content of Changes

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    changes_full = pd.read_sql_query("""
        SELECT 
            change_type,
            current_checksum,
            previous_checksum,
            current_content,
            previous_content,
            similarity_score
        FROM content_change_log
        WHERE detection_run_id = ?
          AND file_name = 'Employee_Handbook.pdf'
        ORDER BY change_type
    """, conn, params=[latest_run])

for idx, row in changes_full.iterrows():
    print("="*100)
    print(f"Change Type: {row['change_type']}")
    print(f"Similarity Score: {row['similarity_score']}")
    print("="*100)
    
    if row['change_type'] == 'NEW_CONTENT':
        print("\n📄 NEW CONTENT:")
        print(f"Checksum: {row['current_checksum']}")
        print("-" * 100)
        print(row['current_content'])
    
    elif row['change_type'] == 'MODIFIED_CONTENT':
        print("\n📄 PREVIOUS VERSION:")
        print(f"Checksum: {row['previous_checksum']}")
        print("-" * 100)
        print(row['previous_content'])
        print("\n📄 CURRENT VERSION:")
        print(f"Checksum: {row['current_checksum']}")
        print("-" * 100)
        print(row['current_content'])
    
    elif row['change_type'] == 'DELETED_CONTENT':
        print("\n📄 DELETED CONTENT:")
        print(f"Checksum: {row['previous_checksum']}")
        print("-" * 100)
        print(row['previous_content'])
    
    elif row['change_type'] == 'UNCHANGED':
        print("\n📄 UNCHANGED CONTENT:")
        print(f"Checksum: {row['current_checksum']}")
        print("-" * 100)
        print(row['current_content'])
    
    print("\n\n")

## 5. Summary Statistics

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    summary_df = pd.read_sql_query("""
        SELECT 
            file_name,
            change_type,
            COUNT(*) as count
        FROM content_change_log
        WHERE detection_run_id = ?
        GROUP BY file_name, change_type
        ORDER BY file_name, change_type
    """, conn, params=[latest_run])

print("\nChange Summary by File:\n")
display(summary_df)

# Overall summary
with sqlite3.connect(DB_PATH) as conn:
    overall_df = pd.read_sql_query("""
        SELECT 
            change_type,
            COUNT(*) as total_count
        FROM content_change_log
        WHERE detection_run_id = ?
        GROUP BY change_type
        ORDER BY change_type
    """, conn, params=[latest_run])

print("\nOverall Change Summary:\n")
display(overall_df)